In [ ]:
LIST_OF_DIRECTORIES = ["output/basfws/ambiguous/coffee", "output/basfws/ambiguous/other", "output/basfws/coffee",
                       "output/basfws/intercrop", "output/basfws/native_no_tree", "output/basfws/nativevege",
                       "output/basfws/other_tree", "output/basfws/pepper", "output/basfws/pine_trees",
                       "output/basfws/rice", "output/basfws/rubber", "output/basfws/seasonal", "output/basfws/tea",
                       "output/basfws/urban", "output/basfws/water", "output/droitefws/ambiguous",
                       "output/droitefws/coffee", "output/droitefws/native_no_tree", "output/droitefws/nativevege",
                       "output/droitefws/other_tree", "output/droitefws/pepper", "output/droitefws/pine_trees",
                       "output/droitefws/rice", "output/droitefws/rubber", "output/droitefws/seasonal", 
                       "output/droitefws/tea", "output/droitefws/urban", "output/droitefws/water", 
                       "output/gauchefws/ambiguous/coffee", "output/gauchefws/ambiguous/pepper",
                       "output/gauchefws/coffee", "output/gauchefws/intercrop", "output/gauchefws/native_no_tree",
                       "output/gauchefws/nativevege", "output/gauchefws/other_tree", "output/gauchefws/pepper",
                       "output/gauchefws/rice", "output/gauchefws/rubber", "output/gauchefws/seasonal",
                       "output/gauchefws/stick_pepper", "output/gauchefws/urban", "output/gauchefws/water",
                       "output/hautfws/ambiguous/coffee", "output/hautfws/ambiguous/other", "output/hautfws/cassava",
                       "output/hautfws/coffee", "output/hautfws/deciduous_forest", "output/hautfws/intercrop",
                       "output/hautfws/native_no_tree", "output/hautfws/nativevege", "output/hautfws/other_tree",
                       "output/hautfws/pepper", "output/hautfws/pine_trees", "output/hautfws/rice", 
                       "output/hautfws/rubber", "output/hautfws/seasonal", "output/hautfws/tea", "output/hautfws/urban",
                       "output/hautfws/water", "output/milieufws/cassava", "output/milieufws/coffee",
                       "output/milieufws/deciduous_forest", "output/milieufws/intercrop", 
                       "output/milieufws/native_no_tree", "output/milieufws/nativevege", "output/milieufws/other_tree",
                       "output/milieufws/pepper", "output/milieufws/rice", "output/milieufws/rubber",
                       "output/milieufws/seasonal", "output/milieufws/urban", "output/milieufws/water"]
COUNTAIN_COFFEE = "coffee"
ZONES = ["basfws", "droitefws", "gauchefws", "hautfws", "milieufws"]
PIXEL_LENGTH = 15
BAND_WIDTH = 9
NB_SPLIT = 5
EPOCH = 1000

# Première étape : charger les données

Importer le nom des données

In [ ]:
import os 
import numpy as np
from matplotlib import pyplot as pl
import random
import rasterio
import os
os.environ['CUDA_VISIBLE_DEVICES']='1'

%matplotlib inline

In [ ]:
def get_all_images_names(directories): 
    images_name = []
    for directory in directories:
        counter = 0
        for filename in os.listdir(directory): 
            if os.path.isfile(directory + "/" + filename):
                images_name.append(directory + "/" + filename)
                counter += 1
            elif os.path.isdir(directory + "/" + filename):
                images_name += get_all_images_names([directory + "/" + filename])
        print(directory, counter, "images.")
    return images_name
images_name = get_all_images_names(LIST_OF_DIRECTORIES)

Importer les images et leur résultat voulu

In [ ]:
def reshape(image):
    """Reformate les données rasterio pour être utilisée par Tensorflow"""
    reshaped_image = []
    for i in range(len(image[0])):
        reshaped_row = []
        for j in range(len(image[0][0])):
            reshaped_cell = []
            
            reshaped_cell.append(image[0][i][j] / 65535)
            reshaped_cell.append(image[1][i][j] / 65535)
            #reshaped_cell.append(image[2][i][j] / 65535)
            reshaped_cell.append(image[3][i][j] / 65535)
            
            reshaped_cell.append(image[4][i][j] / 65535)
            #reshaped_cell.append(image[5][i][j] / 65535)
            reshaped_cell.append(image[6][i][j] / 65535)
            reshaped_cell.append(image[7][i][j] / 65535)
            
            reshaped_cell.append(image[8][i][j] / 65535)
            reshaped_cell.append(image[9][i][j] / 65535)
            #reshaped_cell.append(image[10][i][j] / 65535)
            reshaped_cell.append(image[11][i][j] / 65535)
            
            reshaped_row.append(reshaped_cell)
        reshaped_image.append(reshaped_row)
    return reshaped_image

In [ ]:
images = []
for image_name in images_name:
    z = None
    for zone in ZONES:
        if zone in image_name:
            z = zone
            break
    else:
        sys.exit("unknown zone")
    if COUNTAIN_COFFEE in image_name:
        images.append((np.array(rasterio.open(image_name).read()), 1, z))
    else:
        images.append((np.array(rasterio.open(image_name).read()), 0, z))
print("Longueur initiale : ", len(images))

In [ ]:
def add_all_rotations(images):
    result = []
    for image in images:
        result.append(image)
        for i in range (1, 4):
            rotated_image = []
            for j in range(12): #BAND_WIDTH
                rotated_image.append(np.rot90(image[0][j], i))
            result.append((rotated_image, image[1], image[2]))
    return result

In [ ]:
images = add_all_rotations(images)
random.shuffle(images)

In [ ]:
data = {}
for zone in ZONES:
    data[zone] = {"x":[], "y":[]}
for image in images:
    data[image[2]]["x"].append(reshape(image[0]))
    data[image[2]]["y"].append(image[1])

# Deuxième étape : créer notre cross validation

In [ ]:
from sklearn.model_selection import StratifiedKFold
from sklearn import metrics as me

In [ ]:
kfold = StratifiedKFold(n_splits=NB_SPLIT)

In [ ]:
for key in data:
    data[key]["x"] = np.array(data[key]["x"])
    data[key]["y"] = np.array(data[key]["y"])
    print(np.count_nonzero(data[key]["y"] < 1), len(data[key]["y"]))
    print(np.count_nonzero(data[key]["y"] < 1) / len(data[key]["y"]))

# Troisième étape : entrainer et tester

In [ ]:
def get_fscore_recall_precision(truth, prediction):
    true_positive = 0
    false_negative = 0
    false_positive = 0
    for i in range(len(truth)):
        if truth[i] == 1 and prediction[i] == 1:
            true_positive += 1
        elif truth[i] == 0 and prediction[i] == 1:
            false_positive += 1
        elif truth[i] == 1 and prediction[i] == 0:
            false_negative += 1
    precision = true_positive/(true_positive + false_positive) if (true_positive + false_positive) > 0 else None
    recall = true_positive/(true_positive + false_negative) if (true_positive + false_negative) > 0 else None
    fscore = 2 * recall * precision / (recall + precision) if recall != None and precision != None else None
    return fscore, recall, precision

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten, Dropout, MaxPooling2D

i = 0
for key in data:
    # balance classes
    class_weight = None
    coffee = np.count_nonzero(data[key]["y"] == 1)
    other = len(data[key]["y"]) - coffee
    if other >= coffee:
        class_weight = {0: 1.,1: other / coffee}  
    else:
        class_weight = {0: coffee / other,1: 1.} 
    # create model
    model = Sequential()
    model.add(Conv2D(32, (3,3), activation='relu',  input_shape=(PIXEL_LENGTH,PIXEL_LENGTH,BAND_WIDTH)))
    model.add(MaxPooling2D(pool_size = (2 ,2)))
    model.add(Conv2D(16, (3,3), activation='relu'))
    model.add(MaxPooling2D(pool_size = (2 ,2)))
    model.add(Dropout(0.25))
    model.add(Flatten())
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(1, activation='sigmoid'))
    # Compile model
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # Fit the model
    history = model.fit(data[key]["x"], data[key]["y"], epochs=EPOCH, batch_size=32, class_weight=class_weight)

    print("Trained on", key)
    avg_scores = []
    avg_recall = []
    avg_precision = []
    avg_fscore = [] 
    model.save("model_" + key + ".h5")
    for key2 in data:
        if key2 != key:
            # evaluate the model
            print("Tested on", key2)
            scores = model.evaluate(data[key2]["x"], data[key2]["y"], verbose=0)
            print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))
            avg_scores.append(scores[1] * 100)


            print('Test score:', scores[0])
            print('Test accuracy:', scores[1])

            pred = model.predict_classes(data[key2]["x"])
            f, r, p = get_fscore_recall_precision(data[key2]["y"], pred)
            print('Test recall', r)
            print('Test precision', p)
            print('Test f-score', f)
            avg_recall.append(r * 100)
            avg_precision.append(p * 100)
            avg_fscore.append(f * 100)
            print(me.confusion_matrix(data[key2]["y"], pred))

    print("Visibility %.2f%% (+/- %.2f%%)" % (np.mean(avg_scores), np.std(avg_scores)))
    print("Recall %.2f%% (+/- %.2f%%)" % (np.mean(avg_recall), np.std(avg_recall)))
    print("Precision %.2f%% (+/- %.2f%%)" % (np.mean(avg_precision), np.std(avg_precision)))
    print("F-Score %.2f%% (+/- %.2f%%)" % (np.mean(avg_fscore), np.std(avg_fscore)))